This is demonstration only.

In [3]:
import subprocess

In [4]:
response_id = 137653

In [5]:
cmd = 'python ../scripts/word_count_cli.py -c ../local/big_rds.conf -i %s' % response_id

In [15]:
s = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = s.communicate()

stdout

'A eng utf8 utf8 series series DOC/NOAA/NOS/OCS United States Department Commerce (DOC), National Oceanic Atmospheric Administration (NOAA), National Ocean Service (NOS), Office Coast Survey (OCS) Hydrographic Services Division, IT Specialist x118 NOAA/NOS/OCS/HSD East-West Highway, SSMC3 Silver Spring MD USA hydro.info@noaa.gov pointOfContact pointOfContact ISO 19115-2 Geographic Information Metadata Part Extensions for Imagery Gridded Data ISO E) UNKNOWN Horizontal Datum: Unknown inapplicable Horizontal Datum: Unknown Undetermined vertical datum Vertical Datum: Unknown inapplicable Vertical Datum: Unknown Metadata Extension Information Metadata Extension Information information information H05680A: NOS Hydrographic Survey, publication publication Survey ID: H05680A DOC/NOAA/NOS/HSD United States Department Commerce (DOC), National Oceanic Atmospheric Administration (NOAA), National Ocean Service (NOS), Office Coast Survey (OCS), Hydrographic Surveys Division (HSD) originator originat

In [12]:
stderr

''

In [16]:
len(stdout.split())

1377

In [17]:
stdout.split()[:5]

['A', 'eng', 'utf8', 'utf8', 'series']